In [20]:
import urllib.request, json 
import math
import csv
import requests
import time
from datetime import datetime, timedelta
# import pandas as pd

In [21]:
BASE_ELASTIC_URL = "http://192.168.36.98/mytimes/elasticCommentQuery?sort=desc&appKey=TOI,ET&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED"
BATCH_SIZE_PARAM = 250
# ITER_BUFFER = 2
CSV_FILE_NAME = '/Users/yash.dalmia/elasticComments.csv'
DEFAULT_FIELDS = ['C_T']

In [22]:
def getModifiedUrl(url, from_param=None, size_param=None, startEpoch=None, endEpoch=None):
    result_url = url
    if (from_param!=None and size_param!=None):
        result_url = result_url + "&from=" + str(from_param) + "&size=" + str(size_param)
    if (startEpoch!=None and endEpoch!=None):
        result_url = result_url + "&sDateEpoch=" + str(startEpoch) + "&eDateEpoch=" + str(endEpoch)
    return result_url

In [23]:
def process_paginated_comments_from_elastic(url, size_param = BATCH_SIZE_PARAM, force = False, to_process_till = None):
    
    final_url = getModifiedUrl(url, from_param = 0, size_param = size_param)
    json_raw_response = requests.get(final_url).json()

    total_comment_count = json_raw_response['hits']['total']
    
    ELASTIC_LIMIT = 10000
    if (to_process_till != None and to_process_till < total_comment_count):
        total_comment_count = to_process_till
        
    if (total_comment_count > ELASTIC_LIMIT):
        print("total_comment_count greater than {}: {}".format(ELASTIC_LIMIT, total_comment_count))
        if not force:
            print("aborting")
            return
        else:
            total_comment_count = ELASTIC_LIMIT
    
    num_iters = math.ceil(total_comment_count/size_param)
#     num_iters = 1

    print("total_count : {}".format(total_comment_count))
    print("num_iters : {}".format(num_iters))

    for iter_val in range(0, num_iters, 1):
        print("iteration : %d" %(iter_val))
        from_val = iter_val * size_param
        final_url = getModifiedUrl(url, from_param = from_val, size_param = size_param)
        print('final_url : {}'.format(final_url))
        json_raw_response = requests.get(final_url).json()
        list_comments = [(x['_source']['c_id'], x['_source']['C_T']) for x in json_raw_response['hits']['hits'] if 'C_T' in x['_source']]
        process_list_cmts_elastic(list_comments)   

In [24]:
list_caught_comments = list()

In [25]:
global_counter = 0
def process_list_cmts_elastic(list_comments):
    global global_counter
    for c_id, c_t in list_comments:
        global_counter = global_counter + 1
        process_res = process_per_comment(c_t)
        print("{:>5}. {} : comment id: {}".format(global_counter, process_res, c_id))
        if(process_res):
            list_caught_comments.append((c_id, c_t))

In [26]:
import re
def process_per_comment(c_text):
    c_text = re.sub(" ", "", c_text)
    match_found = re.findall('[0-9]{9,}', c_text)
    return (True if match_found else False)

In [27]:
def get_date_time_hour(hour_delta):
    """
    returns datetime object for start-of-day day_delta days from the current datetime
    """
    dtime = datetime.now()
    dtime = dtime.replace(minute = 0, second = 0, microsecond = 0)
    dtime = dtime + timedelta(hours = hour_delta)
    return dtime

In [28]:
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=asc&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&appKey=ET"
base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED"

batch_write_size = 350

# NET_END_TIME_EPOCH = int(get_date_time_hour(0).timestamp()*1000)
NET_END_TIME_EPOCH = 1544380200000  #10 December, 2018
NUM_DAYS_IN_ONE_ITER = 0.25
NUM_ITERS = 3*4
# NUM_ITERS = 1

# endEpoch = NET_START_TIME_EPOCH
startEpoch = NET_END_TIME_EPOCH

for iter_val in range(0, NUM_ITERS, 1):
    print("\nIter_val : %d" %(iter_val))
    endEpoch = startEpoch
    startEpoch = endEpoch - int(NUM_DAYS_IN_ONE_ITER*24*60*60*1000)
    
#     url = getModifiedUrlDate(base_url, startEpoch, endEpoch)
    url = getModifiedUrl(base_url, startEpoch = startEpoch, endEpoch = endEpoch)
    print("url : " + url)
    process_paginated_comments_from_elastic(url, size_param = batch_write_size)

print("\nnet startEpoch : ", startEpoch)
print("net endEpoch : ", NET_END_TIME_EPOCH)


Iter_val : 0
url : http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED&sDateEpoch=1544358600000&eDateEpoch=1544380200000
total_count : 4386
num_iters : 13
iteration : 0
final_url : http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED&sDateEpoch=1544358600000&eDateEpoch=1544380200000&from=0&size=350
    1. False : comment id: 2448469954
    2. False : comment id: 2448469952
    3. False : comment id: 2448469946
    4. False : comment id: 2448469904
    5. False : comment id: 2448469942
    6. False : comment id: 2448469940
    7. False : comment id: 2448469901
    8. False : comment id: 2448469895
    9. False : comment id: 2448469933
   10. False : comment id: 2448469885
   11. False : comment id: 2448469921
   12. False : comment id: 2448469919
   13. False : comment id: 2448469869
   14. False : comment id: 2448469848
   15. False : comment id: 2448469841
   16. False : comment

  347. False : comment id: 2448466135
  348. False : comment id: 2448466097
  349. False : comment id: 2448466116
  350. False : comment id: 2448466114
  351. False : comment id: 2448466079
  352. False : comment id: 2448466109
  353. False : comment id: 2448466056
  354. False : comment id: 2448466055
  355. False : comment id: 2448466074
  356. False : comment id: 2448466051
  357. False : comment id: 2448466049
  358. False : comment id: 2448466063
  359. False : comment id: 2448466041
  360. False : comment id: 2448466038
  361. False : comment id: 2448466006
  362. False : comment id: 2448465997
  363. False : comment id: 2448466021
  364. False : comment id: 2448465990
  365. False : comment id: 2448465987
  366. False : comment id: 2448465975
  367. False : comment id: 2448465954
  368. False : comment id: 2448465945
  369. False : comment id: 2448465940
  370. False : comment id: 2448465908
  371. False : comment id: 2448465898
  372. False : comment id: 2448465897
  373. False

  692. False : comment id: 2448462741
  693. False : comment id: 2448462720
  694. False : comment id: 2448462710
  695. False : comment id: 2448462688
  696. False : comment id: 2448462685
  697. False : comment id: 2448462681
  698. False : comment id: 2448462642
  699. False : comment id: 2448462641
  700. False : comment id: 2448462670
  701. False : comment id: 2448462639
  702. False : comment id: 2448462608
  703. False : comment id: 2448462615
  704. False : comment id: 2448462594
  705. False : comment id: 2448462589
  706. False : comment id: 2448462554
  707. False : comment id: 2448462552
  708. False : comment id: 2448462580
  709. False : comment id: 2448462550
  710. False : comment id: 2448462569
  711. False : comment id: 2448462501
  712. False : comment id: 2448462497
  713. False : comment id: 2448462520
  714. False : comment id: 2448462493
  715. False : comment id: 2448462487
  716. False : comment id: 2448462469
  717. False : comment id: 2448462443
  718. False

 1038. False : comment id: 2448458946
 1039. False : comment id: 2448458904
 1040. False : comment id: 2448458887
 1041. False : comment id: 2448458921
 1042. False : comment id: 2448458872
 1043. False : comment id: 2448458917
 1044. False : comment id: 2448458916
 1045. False : comment id: 2448458911
 1046. False : comment id: 2448458851
 1047. False : comment id: 2448458806
 1048. False : comment id: 2448458845
 1049. False : comment id: 2448458800
 1050. False : comment id: 2448458843
 1051. False : comment id: 2448458797
 1052. False : comment id: 2448458793
 1053. False : comment id: 2448458703
 1054. False : comment id: 2448458745
 1055. False : comment id: 2448458738
 1056. False : comment id: 2448458687
 1057. False : comment id: 2448458717
 1058. False : comment id: 2448458651
 1059. False : comment id: 2448458645
 1060. False : comment id: 2448458639
 1061. False : comment id: 2448458636
 1062. False : comment id: 2448458581
 1063. False : comment id: 2448458626
 1064. False

 1383. False : comment id: 2448455146
 1384. False : comment id: 2448455162
 1385. False : comment id: 2448455130
 1386. False : comment id: 2448455117
 1387. False : comment id: 2448455083
 1388. False : comment id: 2448455082
 1389. False : comment id: 2448455049
 1390. False : comment id: 2448455043
 1391. False : comment id: 2448455042
 1392. False : comment id: 2448455069
 1393. False : comment id: 2448455033
 1394. False : comment id: 2448455007
 1395. False : comment id: 2448455020
 1396. False : comment id: 2448455019
 1397. False : comment id: 2448454985
 1398. False : comment id: 2448454956
 1399. False : comment id: 2448454984
 1400. False : comment id: 2448454954
 1401. False : comment id: 2448454947
 1402. False : comment id: 2448454945
 1403. False : comment id: 2448454976
 1404. False : comment id: 2448454970
 1405. False : comment id: 2448454967
 1406. False : comment id: 2448454931
 1407. False : comment id: 2448454917
 1408. False : comment id: 2448454895
 1409. False

...

40243. False : comment id: 2447861020
40244. False : comment id: 2447860953
40245. False : comment id: 2447860961
40246. False : comment id: 2447860944
40247. False : comment id: 2447860935
40248. False : comment id: 2447860923
40249. False : comment id: 2447860909
40250. False : comment id: 2447860863
40251. False : comment id: 2447860850
40252. False : comment id: 2447860807
40253. False : comment id: 2447860801
40254. False : comment id: 2447860842
40255. False : comment id: 2447860839
40256. False : comment id: 2447860838
40257. False : comment id: 2447860814
40258. False : comment id: 2447860770
40259. False : comment id: 2447860757
40260. False : comment id: 2447860768
40261. False : comment id: 2447860761
40262. False : comment id: 2447860706
40263. False : comment id: 2447860744
40264. False : comment id: 2447860703
40265. False : comment id: 2447860698
40266. False : comment id: 2447860733
40267. False : comment id: 2447860732
40268. False : comment id: 2447860727
40269. False

40700. False : comment id: 2447855189
40701. False : comment id: 2447855148
40702. False : comment id: 2447855145
40703. False : comment id: 2447855180
40704. False : comment id: 2447855166
40705. False : comment id: 2447855107
40706. False : comment id: 2447855104
40707. False : comment id: 2447855118
40708. False : comment id: 2447855045
40709. False : comment id: 2447855076
40710. False : comment id: 2447855073
40711. False : comment id: 2447855032
40712. False : comment id: 2447855016
40713. False : comment id: 2447854998
40714. False : comment id: 2447854986
40715. False : comment id: 2447854984
40716. False : comment id: 2447854943
40717. False : comment id: 2447854930
40718. False : comment id: 2447854965
40719. False : comment id: 2447854857
40720. False : comment id: 2447854855
40721. False : comment id: 2447854847
40722. False : comment id: 2447854809
40723. False : comment id: 2447854792
40724. False : comment id: 2447854746
40725. False : comment id: 2447854743
40726. False

41050. False : comment id: 2447849488
41051. False : comment id: 2447849478
41052. False : comment id: 2447849473
41053. False : comment id: 2447849472
41054. False : comment id: 2447849441
41055. False : comment id: 2447849461
41056. False : comment id: 2447849437
41057. False : comment id: 2447849399
41058. False : comment id: 2447849398
41059. False : comment id: 2447849397
41060. False : comment id: 2447849391
41061. False : comment id: 2447849381
41062. False : comment id: 2447849409
41063. False : comment id: 2447849300
41064. False : comment id: 2447849324
41065. False : comment id: 2447849294
41066. False : comment id: 2447849315
41067. False : comment id: 2447849270
41068. False : comment id: 2447849245
41069. False : comment id: 2447849261
41070. False : comment id: 2447849230
41071. False : comment id: 2447849186
41072. False : comment id: 2447849209
41073. False : comment id: 2447849147
41074. False : comment id: 2447849099
41075. False : comment id: 2447849112
41076. False

41393. False : comment id: 2447842681
41394. False : comment id: 2447842679
41395. False : comment id: 2447842603
41396. False : comment id: 2447842593
41397. False : comment id: 2447842536
41398. False : comment id: 2447842502
41399. False : comment id: 2447842525
41400. False : comment id: 2447842493
41401. False : comment id: 2447842509
41402. False : comment id: 2447842446
41403. False : comment id: 2447842461
41404. False : comment id: 2447842421
41405. False : comment id: 2447842416
41406. False : comment id: 2447842368
41407. False : comment id: 2447842294
41408. False : comment id: 2447842288
41409. False : comment id: 2447842273
41410. False : comment id: 2447842247
41411. False : comment id: 2447842268
41412. False : comment id: 2447842194
41413. False : comment id: 2447842192
41414. False : comment id: 2447842212
41415. False : comment id: 2447842143
41416. False : comment id: 2447842160
41417. False : comment id: 2447842108
41418. False : comment id: 2447842132
41419. False

41726. False : comment id: 2447837111
41727. False : comment id: 2447837045
41728. False : comment id: 2447837040
41729. False : comment id: 2447837069
41730. False : comment id: 2447837063
41731. False : comment id: 2447837020
41732. False : comment id: 2447836974
41733. False : comment id: 2447836926
41734. False : comment id: 2447836897
41735. False : comment id: 2447836848
41736. False : comment id: 2447836886
41737. False : comment id: 2447836885
41738. False : comment id: 2447836844
41739. False : comment id: 2447836841
41740. False : comment id: 2447836834
41741. False : comment id: 2447836806
41742. False : comment id: 2447836814
41743. False : comment id: 2447836799
41744. False : comment id: 2447836657
41745. False : comment id: 2447836620
41746. False : comment id: 2447836660
41747. False : comment id: 2447836659
41748. False : comment id: 2447836614
41749. False : comment id: 2447836544
41750. False : comment id: 2447836541
41751. False : comment id: 2447836566
41752. False

In [37]:
print(len(list_caught_comments))

39


In [30]:
for c_id, c_t in list_caught_comments:
    print(c_t)

6397663738
sahjad bahi.  6397663738
300000000
3121010068105013641560807196
111111....4444555678...
It''s time we do similar thing in India against feku, it will be 3100000000 people
10520768168 registration no logon nahi ho raha koi bata do please
Registration No. 10502654922
10513924881
This is about Dr korofo a spellcaster/herbal Dr that just get rid f the HIV Virus and Cancer that i have being suffering from, I came in contact with Dr korofo on Facebook and i explained my predicaments to him and He decided to help me with his medication. I only used the medication for 3 weeks and i went for medical check-up as instructed by the great Korofo and the virus was no where to be found. This looks like a dream to me because i have being using other medication but none was able to cure me, i almost believed that HIV Virus had no cure according to the scientist but Dr korofo made me believe that there is treatment, prevention and cure for HIV Virus. Am now free from the HIV virus and Cancer 

In [31]:
for c_id, c_t in list_caught_comments:
    print(c_t)

6397663738
sahjad bahi.  6397663738
300000000
3121010068105013641560807196
111111....4444555678...
It''s time we do similar thing in India against feku, it will be 3100000000 people
10520768168 registration no logon nahi ho raha koi bata do please
Registration No. 10502654922
10513924881
This is about Dr korofo a spellcaster/herbal Dr that just get rid f the HIV Virus and Cancer that i have being suffering from, I came in contact with Dr korofo on Facebook and i explained my predicaments to him and He decided to help me with his medication. I only used the medication for 3 weeks and i went for medical check-up as instructed by the great Korofo and the virus was no where to be found. This looks like a dream to me because i have being using other medication but none was able to cure me, i almost believed that HIV Virus had no cure according to the scientist but Dr korofo made me believe that there is treatment, prevention and cure for HIV Virus. Am now free from the HIV virus and Cancer 

In [32]:
list_caught_comments

[(2448439140, '6397663738'),
 (2448421059, 'sahjad bahi.  6397663738'),
 (2448402428, '300000000'),
 (2448358591, '3121010068105013641560807196'),
 (2448351363, '111111....4444555678...'),
 (2448297384,
  "It''s time we do similar thing in India against feku, it will be 3100000000 people"),
 (2448275634,
  '10520768168 registration no logon nahi ho raha koi bata do please'),
 (2448272139, 'Registration No. 10502654922'),
 (2448268019, '10513924881'),
 (2448232933,
  'This is about Dr korofo a spellcaster/herbal Dr that just get rid f the HIV Virus and Cancer that i have being suffering from, I came in contact with Dr korofo on Facebook and i explained my predicaments to him and He decided to help me with his medication. I only used the medication for 3 weeks and i went for medical check-up as instructed by the great Korofo and the virus was no where to be found. This looks like a dream to me because i have being using other medication but none was able to cure me, i almost believed tha

In [33]:
import pandas as pd

In [34]:
df = pd.DataFrame(list_caught_comments, columns = ['c_id', 'c_t'])

In [35]:
df.to_csv('/Users/yash.dalmia/phone_num_regex_6_9_dec_18.csv', index=False)

IndexError: single positional indexer is out-of-bounds